# Ajinomoto Project Data Unification
Here we pull in all data types and create a unified data frame with all strain information. This will then be converted into a format that is usable by EDD.

**Todo:**

1. [X] Figure out Ways Representing Intiutive Strain Metadata and add to sheet! -- The Result should be no Redundant Indecies!
2. [X] Add RBS Data to ALL Strains!
3. [X] Check to make sure all expected information is present! (No NaNs where we have information)

In [1]:
import pandas as pd
from IPython.display import display

STANDARD_IPTG = 0.1 # mM concentration

## Cycle 1 Import

In [2]:
#Cycle 1 Data Frame Unification 
df = pd.read_csv('data/ajinomoto_oct23.csv')

In [3]:
#Create a Dataframe of Protein and Metabolite Single Time Point Measurement
measurement_df = pd.pivot_table(df, values=['0.0','27.0'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
measurement_df.columns = measurement_df.columns.droplevel(0)

#Change Proteomics to SwissProt Identifiers
swiss_prot_name = {'A1U2T0_MARHV':'tr|A1U2T0|A1U2T0_MARHV',
                   'A1U3L3_MARHV':'tr|A1U3L3|A1U3L3_MARHV',
                   'ACR1_ACIAD':'sp|Q6F7B8|ACR1_ACIAD',
                   'AHR_ECOLI':'sp|P27250|AHR_ECOLI',
                   'FATB_UMBCA':'sp|Q41635|FATB_UMBCA',
                   'LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'
                  }

chem_names = {'Dodecanoate (n-C12:0)':'dodecan-1-ol',
              'Hexadecanoate (n-C16:0)':'hexadecan-1-ol',
              'Octadecanoate (n-C18:0)':'octadecan-1-ol',
              'Tetradecanoate (n-C14:0)':'tetradecan-1-ol'
             }

#Change GCMS To Correct Naming Convention
measurement_df.rename(columns = swiss_prot_name, inplace = True, level=1)
measurement_df.rename(columns = chem_names, inplace = True, level=1)

#display(measurement_df)

In [4]:
#Load Design data from ICE
design_df = df[['Line Name','Strain']]
design_df = design_df.groupby(['Line Name','Strain']).size().reset_index().set_index('Line Name')[['Strain']]

#Add Batch Number, Tak Number, and Replicate
identifiers = [index.split('-') for index in design_df.index]
identifiers = [[int(identifier[0][1]), int(identifier[1]), int(identifier[2])] for identifier in identifiers]
design_df[['Batch','Strain','Replicate']] = pd.DataFrame(identifiers, index=design_df.index)
design_df['Cycle']=1

design_df.loc[design_df['Strain']==9017,'Cycle'] = 0

#Add Design Information (RBS Strength To Each Design For Each Protein)
rbs_df = pd.read_csv('data/design.csv')
rbs_df.rename(columns = {'Tak':'Strain'}, inplace = True)


#display(rbs_df)
indecies = design_df.index
design_df = pd.merge(design_df,rbs_df,how='left',on="Strain")
design_df.index = indecies

#Add Pathway Variant ID
#design_df['Pathway Variant'] = 0
#pathway_genes = ['Maqu_2507','Maqu_2220','acr1']
#for i,pathway_gene in enumerate(pathway_genes):
    #Find Indecies Where Pathway Gene is present
#    design_df['Pathway Variant'].loc[~design_df[pathway_gene].isnull()] = i+1

design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])
#display(design_df)

In [5]:
#Load Metabolomics Data
metabolomics_df = pd.read_csv('data/metabolomics.csv')
columns = ['Decanoic acid', 'Lauric acid', 'Myristic acid', 'Palmitic acid',
           'Stearic acid', 'Palmitoleic acid', 'Oleic acid']# 'Dodecanoyl-CoA',
           #'Tetradecanoyl-CoA', 'Hexadecanoyl-CoA', 'Stearoyl-CoA', 'Acetyl-CoA']
augmented_cols = pd.MultiIndex.from_tuples([('LC-MS',col) for col in columns])
metabolomics_df = metabolomics_df.set_index('EDD Line Name')
metabolomics_df = metabolomics_df[columns]
metabolomics_df.columns = augmented_cols
metabolomics_df = metabolomics_df.fillna(0)

In [6]:
#Load Nims Data
nims_df = pd.read_csv('data/nims.csv')
nims_averaged_df = nims_df.groupby('Unnamed: 0').mean()
nims_averaged_df.columns = pd.MultiIndex.from_tuples([('NIMS',col) for col in nims_averaged_df.columns])

In [7]:
df = pd.merge(measurement_df,design_df,how='left',left_index=True,right_index=True)
df = pd.merge(df,metabolomics_df,how='left',left_index=True,right_index=True)
df = pd.merge(df,nims_averaged_df,how='left',left_index=True,right_index=True)

In [8]:
#Load Translation Initiation Rates Associated with Each RBS.
rbs_df = pd.read_csv('data/rbs.csv')
#Define Function to associate Translation Initiation Rate with Each Gene
def find_tir(gene,value_column='Predicted TIR'):
    def f(row):
        rbs_strength = row[('Metadata',gene)]
        if isinstance(rbs_strength,str):
            return rbs_df[(rbs_df['Gene'] == gene)&(rbs_df['Strength'] == row[('Metadata',gene)])][value_column].values[0]
        else:
            return 0        
    return f

genes = ['UcFatB1','FadD','Maqu_2507','Maqu_2220','acr1','yjgB']
for gene in genes:
    df[('TIR',gene)] = df.apply(find_tir(gene),axis=1)
    df[('EMOPEC',gene)] = df.apply(find_tir(gene,value_column='EMOPEC'),axis=1)
    measurement_df[('TIR',gene)] = df.apply(find_tir(gene),axis=1)
    measurement_df[('EMOPEC',gene)] = df.apply(find_tir(gene,value_column='EMOPEC'),axis=1)

replacement = {'UcFatB1':'sp|Q41635|FATB_UMBCA',
               'FadD':'sp|P69451|LCFA_ECOLI',
               'Maqu_2507':'tr|A1U3L3|A1U3L3_MARHV',
               'Maqu_2220':'tr|A1U2T0|A1U2T0_MARHV',
               'acr1':'sp|Q6F7B8|ACR1_ACIAD',
               'yjgB':'sp|P27250|AHR_ECOLI'}

df.rename(columns = replacement, inplace = True, level=1)

In [9]:
#drop some metadata columns
proteins = ['sp|P27250|AHR_ECOLI','sp|P69451|LCFA_ECOLI','sp|Q41635|FATB_UMBCA','sp|Q6F7B8|ACR1_ACIAD','tr|A1U2T0|A1U2T0_MARHV','tr|A1U3L3|A1U3L3_MARHV']
for col in proteins:
    df = df.drop(('Metadata',col),axis=1)
#display(df['Metadata'])
df[('Metadata','IPTG')] = STANDARD_IPTG

## Import Cycle 2 Data

In [10]:
#Import Batch3
#Import Data Files
cycle2_df = pd.read_csv('data/Cycle2Batch3.csv')
cycle2_df =  pd.pivot_table(cycle2_df, values=['27.0'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
cycle2_df.columns = cycle2_df.columns.droplevel(0)

#Add Metadata
design_df = pd.DataFrame()
identifiers = [index.split('-') for index in cycle2_df.index]
identifiers = [[3,identifier[0],int(identifier[1][1:])] for identifier in identifiers]

import re

def id_to_metadata(ID):
    cycle,strain = re.split(r"[_, ]", ID)
    cycle = int(cycle[1])
    strain = int(re.sub('[^0-9]','', strain))
    
    if strain==9017:
        cycle = 0
    return cycle,strain
    
design_df['Cycle'] = 0
design_df['Strain'] = 0
design_df['IPTG'] = STANDARD_IPTG
design_df[['Batch','ID','Replicate']] = pd.DataFrame(identifiers, index=cycle2_df.index)
design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])
design_df[('Metadata','Cycle')] = design_df[('Metadata','ID')].apply(lambda x: id_to_metadata(x)[0])
design_df[('Metadata','Strain')] = design_df[('Metadata','ID')].apply(lambda x: id_to_metadata(x)[1])
design_df = design_df.drop(('Metadata', 'ID'), axis = 1)

#Merge DataFrame
cycle2_df = pd.merge(cycle2_df,design_df,how='left',right_index=True,left_index=True)

#Convert Targeted Proteomics to Swiss Prot
cycle2_df.rename(columns = {'AHR_ECOLI':'sp|P27250|AHR_ECOLI','LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'}, inplace = True, level=1)


In [11]:
def find_cycle(id_string):
    id_elements = id_string.split('_')
    if len(id_elements) == 1:
        return 0
    else:
        try:
            return int(id_elements[1][1])
        except:
            return 1
    
def find_intuitive_mods(id_string):
    id_elements = id_string.split('_')
    try:
        int(id_elements[1][1])
        return float('NaN')
    except:
        if len(id_elements) == 1:
            return float('NaN')
        else:
            return id_elements[1]


#Import Batch4
batch4_df = pd.read_csv('data/Cycle2Batch4.csv')
batch4_df =  pd.pivot_table(batch4_df, values=['27.0'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
batch4_df.columns = batch4_df.columns.droplevel(0)

#Add Metadata
design_df = pd.DataFrame()
identifiers = [index.split('-') for index in batch4_df.index]
identifiers = [[int(identifier[1][1:]),identifier[2],int(identifier[3][1:])] for identifier in identifiers]
design_df[['Batch','ID','Replicate']] = pd.DataFrame(identifiers, index=batch4_df.index)
display(design_df)


design_df['Cycle'] = design_df['ID'].apply(find_cycle)
design_df['Strain'] = design_df['ID'].apply(lambda x: int(x.split('_')[0]))
design_df['Notes'] = design_df['ID'].apply(find_intuitive_mods)
design_df['IPTG'] = STANDARD_IPTG
design_df = design_df.drop('ID',axis=1)
design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])

batch4_df = pd.merge(batch4_df,design_df,how='left',right_index=True,left_index=True)
batch4_df.rename(columns = {'AHR_ECOLI':'sp|P27250|AHR_ECOLI','LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'}, inplace = True, level=1)
batch4_df[('Metadata','IPTG')] = STANDARD_IPTG
display(batch4_df)

,Batch,ID,Replicate
Line Name,,,
c2-b4-01_c2-R1,4,01_c2,1
c2-b4-01_c2-R2,4,01_c2,2
c2-b4-01_c2-R3,4,01_c2,3
c2-b4-02_c2-R1,4,02_c2,1
c2-b4-02_c2-R2,4,02_c2,2
c2-b4-02_c2-R3,4,02_c2,3
c2-b4-04_c2-R1,4,04_c2,1
c2-b4-04_c2-R2,4,04_c2,2
c2-b4-04_c2-R3,4,04_c2,3


Protocol Name                   GC-MS                                \
Measurement Type         dodecan-1-ol hexadecan-1-ol octadecan-1-ol   
Line Name                                                             
c2-b4-01_c2-R1                0.33696        0.07229        0.08122   
c2-b4-01_c2-R2                0.35211        0.07328        0.00000   
c2-b4-01_c2-R3                0.37076        0.07394        0.00000   
c2-b4-02_c2-R1                0.00000        0.09023        0.08115   
c2-b4-02_c2-R2                0.00000        0.08794        0.08138   
c2-b4-02_c2-R3                0.00000        0.09060        0.08364   
c2-b4-04_c2-R1                0.24434        0.07575        0.00000   
c2-b4-04_c2-R2                0.25846        0.07463        0.00000   
c2-b4-04_c2-R3                0.27516        0.07606        0.00000   
c2-b4-11_c1-R1                0.68394        0.07217        0.08350   
c2-b4-11_c1-R2                0.69810        0.07187        0.00000   
c2-b4-11_c1-R3                0.67880        0.07124        0.08357   
c2-b4-11_c2-R1                0.16912        0.00000        0.08137   
c2-b4-11_c2-R2                0.17383        0.06896        0.00000   
c2-b4-11_c2-R3                0.20522        0.06904        0.08102   
c2-b4-14_c2-R1                0.67963        0.00000        0.00000   
c2-b4-14_c2-R2                0.65743        0.00000        0.00000   
c2-b4-14_c2-R3                0.63307        0.06882        0.00000   
c2-b4-24_c2-R1                0.00000        0.06989        0.08223   
c2-b4-24_c2-R2                0.00000        0.06899        0.00000   
c2-b4-24_c2-R3                0.00000        0.06963        0.00000   
c2-b4-32_delPoxB-R1           0.09215        0.10731        0.13531   
c2-b4-32_delPoxB-R2           0.06605        0.09726        0.15003   
c2-b4-32_delPoxB-R3           0.05544        0.10300        0.17131   
c2-b4-32_delPta-R1            0.09751        0.15676        0.16977   
c2-b4-32_delPta-R2            0.10366        0.11327        0.19477   
c2-b4-32_delPta-R3            0.08471        0.10078        0.17669   
c2-b4-32_delPtaelPoxB-R1      0.00000        0.07085        0.00000   
c2-b4-32_delPtaelPoxB-R2      0.00000        0.00000        0.08151   
c2-b4-32_delPtaelPoxB-R3      0.00000        0.00000        0.00000   
c2-b4-9017-R1                 0.00000        0.74598        0.41292   
c2-b4-9017-R2                 0.00000        0.72986        0.38752   
c2-b4-9017-R3                 0.00000        0.73024        0.39947   

Protocol Name                                HPLC                      \
Measurement Type         tetradecan-1-ol  Acetate D-Glucose D-Lactate   
Line Name                                                               
c2-b4-01_c2-R1                   0.02741  0.80379   1.45424   0.09455   
c2-b4-01_c2-R2                   0.02959  1.08644   1.39750   0.06879   
c2-b4-01_c2-R3                   0.02739  1.00452   0.83203   0.06237   
c2-b4-02_c2-R1                   0.02160  1.92526   4.15227   0.03117   
c2-b4-02_c2-R2                   0.02139  2.10107   4.34346   0.04417   
c2-b4-02_c2-R3                   0.02255  2.13015   4.48272   0.04132   
c2-b4-04_c2-R1                   0.02561  1.56070   3.21502   0.05780   
c2-b4-04_c2-R2                   0.02766  1.51547   2.80612   0.06661   
c2-b4-04_c2-R3                   0.02661  1.29715   1.59910   0.05576   
c2-b4-11_c1-R1                   0.04607  0.00000   0.00000   0.00000   
c2-b4-11_c1-R2                   0.04508  0.00000   0.00000   0.00000   
c2-b4-11_c1-R3                   0.04443  0.00000   0.00000   0.00000   
c2-b4-11_c2-R1                   0.02279  1.16211  13.79359   0.09434   
c2-b4-11_c2-R2                   0.02420  1.26488  13.13254   0.09861   
c2-b4-11_c2-R3                   0.02441  1.23885  12.26380   0.11694   
c2-b4-14_c2-R1                   0.04432  0.00000   0.00000   0.00000   
c2-b4-14_c2-R2                   0.04219  0.00000   0.00

In [12]:
#Import Batch5
batch5_df = pd.read_csv('data/Cycle2Batch5.csv')
batch5_df =  pd.pivot_table(batch5_df, values=['27.0'], index=['Line Name'], columns=['Protocol Name','Measurement Type']).fillna(0)
batch5_df.columns = batch5_df.columns.droplevel(0)

#display(batch5_df)

#Add Metadata
design_df = pd.DataFrame()
identifiers = [index.split('-') for index in batch5_df.index]
identifiers = [[int(identifier[1][1:]),identifier[2],int(identifier[3][1:])] for identifier in identifiers]
design_df[['Batch','ID','Replicate']] = pd.DataFrame(identifiers, index=batch5_df.index)
design_df['Strain'] = design_df['ID'].apply(lambda x: int(x.split('_')[0]))
#display(design_df['ID'].apply(lambda x: x.split('_')))

def extract_cycle(id_string):
    id_elements = id_string.split('_')
    if len(id_elements) == 1:
        return 0
    else:
        return int(id_elements[1][1])

def get_iptg(id_string):
    id_elements = id_string.split('_')
    if len(id_elements) == 3:
        return float(id_elements[2][:3])
    else:
        return STANDARD_IPTG

design_df['Cycle'] = design_df['ID'].apply(extract_cycle)
design_df['IPTG'] = design_df['ID'].apply(get_iptg)

design_df = design_df.drop('ID',axis=1)
design_df.columns = pd.MultiIndex.from_tuples([('Metadata',cname) for cname in design_df.columns])

batch5_df = pd.merge(batch5_df,design_df,how='left',right_index=True,left_index=True)
batch5_df.rename(columns = {'AHR_ECOLI':'sp|P27250|AHR_ECOLI','LCFA_ECOLI':'sp|P69451|LCFA_ECOLI'}, inplace = True, level=1)
display(batch5_df)

Protocol Name                   GC-MS                                \
Measurement Type         dodecan-1-ol hexadecan-1-ol octadecan-1-ol   
Line Name                                                             
c2-b5-01_c2_0.5mMiptg-R1      0.27883        0.04932        0.09944   
c2-b5-01_c2_0.5mMiptg-R2      0.28538        0.05151        0.10024   
c2-b5-01_c2_0.5mMiptg-R3      0.27344        0.05008        0.09968   
c2-b5-01_c2_1.0mMiptg-R1      0.30908        0.05033        0.10001   
c2-b5-01_c2_1.0mMiptg-R2      0.31091        0.05055        0.09997   
c2-b5-01_c2_1.0mMiptg-R3      0.29577        0.04920        0.09994   
c2-b5-02_c2_0.5mMiptg-R1      0.00000        0.06280        0.09987   
c2-b5-02_c2_0.5mMiptg-R2      0.00000        0.06506        0.09957   
c2-b5-02_c2_0.5mMiptg-R3      0.00000        0.06655        0.09872   
c2-b5-02_c2_1.0mMiptg-R1      0.00000        0.06025        0.09871   
c2-b5-02_c2_1.0mMiptg-R2      0.00000        0.06254        0.09848   
c2-b5-02_c2_1.0mMiptg-R3      0.00579        0.06265        0.09858   
c2-b5-03_c2-R1                0.00000        0.07227        0.10016   
c2-b5-03_c2-R2                0.00000        0.08256        0.10049   
c2-b5-03_c2-R3                0.00000        0.06460        0.09968   
c2-b5-03_c2_0.5mMiptg-R1      0.00000        0.06382        0.09846   
c2-b5-03_c2_0.5mMiptg-R2      0.00000        0.06276        0.09953   
c2-b5-03_c2_0.5mMiptg-R3      0.00000        0.06168        0.09855   
c2-b5-03_c2_1.0mMiptg-R1      0.00000        0.05985        0.09831   
c2-b5-03_c2_1.0mMiptg-R2      0.00000        0.06192        0.09943   
c2-b5-03_c2_1.0mMiptg-R3      0.00000        0.06243        0.09903   
c2-b5-04_c2_0.5mMiptg-R1      0.24766        0.05359        0.09996   
c2-b5-04_c2_0.5mMiptg-R2      0.24679        0.05082        0.09961   
c2-b5-04_c2_0.5mMiptg-R3      0.26410        0.05461        0.09982   
c2-b5-04_c2_1.0mMiptg-R1      0.23005        0.05175        0.09830   
c2-b5-04_c2_1.0mMiptg-R2      0.24922        0.05032        0.09907   
c2-b5-04_c2_1.0mMiptg-R3      0.27461        0.05198        0.09962   
c2-b5-07_c2-R1                0.93985        0.07363        0.12016   
c2-b5-07_c2-R2                0.92886        0.07197        0.11758   
c2-b5-07_c2-R3                0.94019        0.07101        0.11749   
c2-b5-09_c2-R1                0.14078        0.04405        0.09586   
c2-b5-09_c2-R2                0.15048        0.00000        0.09586   
c2-b5-09_c2-R3                0.14510        0.00000        0.00000   
c2-b5-11_c1-R1                0.72920        0.08681        0.12353   
c2-b5-11_c1-R2                0.74988        0.07663        0.11704   
c2-b5-11_c1-R3                0.73598        0.07596        0.12018   
c2-b5-11_c1_0.5mMiptg-R1      0.65291        0.04964        0.10754   
c2-b5-11_c1_0.5mMiptg-R2      0.66226        0.05041        0.10677   
c2-b5-11_c1_0.5mMiptg-R3      0.64032        0.05021        0.10504   
c2-b5-18_c2-R1                0.03711        0.18792        0.17587   
c2-b5-18_c2-R2                0.01411        0.12519        0.15155   
c2-b5-18_c2-R3                0.01207        0.18738        0.18380   
c2-b5-9017-R1                 0.00000        0.77992        0.49173   
c2-b5-9017-R2                 0.00000        0.73237        0.48453   
c2-b5-9017-R3                 0.00000        0.71551        0.46860   

Protocol Name                                HPLC                      \
Measurement Type         tetradecan-1-ol  Acetate D-Glucose D-Lactate   
Line Name                                                               
c2-b5-01_c2_0.5mMiptg-R1         0.01754  1.76605   1.17840       0.0   
c2-b5-01_c2_0.5mMiptg-R2         0.02072  1.89154   1.26822       0.0   
c2-b5-01_c2_0.5mMiptg-R3         0.01916  1.63942   1.10380       0.0   
c2-b5-01_c2_1.0mMiptg-R1         0.02151  1.26715   0.47127       0.0   
c2-b5-01_c2_1.0mMiptg-R2         0.02156  1.31610   0.69861       0.0   
c2-b5-0

In [13]:
#Merge Together Both Batch 3 & 4
import numpy as np
cycle2_df = pd.concat([cycle2_df,batch4_df,batch5_df])

cycle2_df[('Metadata','IPTG')].loc[np.isnan(cycle2_df[('Metadata','IPTG')])] = STANDARD_IPTG
display(cycle2_df)

/Users/zakcostello/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Protocol Name                   GC-MS                                \
Measurement Type         dodecan-1-ol hexadecan-1-ol octadecan-1-ol   
Line Name                                                             
c1_tak11-R1                   0.63266        0.03313        0.05071   
c1_tak11-R2                   0.61611        0.03219        0.05259   
c1_tak11-R3                   0.66785        0.03024        0.05256   
c1_tak32-R1                   0.15548        0.13168        0.17328   
c1_tak32-R2                   0.15131        0.13126        0.17495   
c1_tak32-R3                   0.13764        0.12044        0.16247   
c2b3 9017-R1                  0.00000        0.75034        0.37241   
c2b3 9017-R2                  0.00000        0.71846        0.34175   
c2b3 9017-R3                  0.00000        0.66553        0.31377   
c2b3_10-R1                    0.28390        0.02503        0.04509   
c2b3_10-R2                    0.30081        0.02420        0.04466   
c2b3_10-R3                    0.28445        0.02413        0.04475   
c2b3_11-R1                    0.27570        0.02377        0.04271   
c2b3_11-R2                    0.33594        0.02479        0.04401   
c2b3_11-R3                    0.35072        0.02571        0.04395   
c2b3_15-R1                    0.04874        0.08111        0.09171   
c2b3_15-R2                    0.05014        0.08146        0.09670   
c2b3_15-R3                    0.04472        0.07332        0.09461   
c2b3_16-R1                    0.00735        0.05042        0.06824   
c2b3_16-R2                    0.00384        0.06946        0.06516   
c2b3_16-R3                    0.00538        0.05593        0.07146   
c2b3_17-R1                    0.00041        0.09912        0.07567   
c2b3_17-R2                    0.00000        0.05677        0.07725   
c2b3_17-R3                    0.00000        0.05201        0.07308   
c2b3_19-R1                    0.00000        0.03830        0.05479   
c2b3_19-R2                    0.00000        0.05436        0.05241   
c2b3_19-R3                    0.00000        0.03415        0.05316   
c2b3_20-R1                    0.07021        0.06538        0.10253   
c2b3_20-R2                    0.06135        0.06137        0.09761   
c2b3_20-R3                    0.06716        0.06293        0.10145   
...                               ...            ...            ...   
c2-b5-03_c2_0.5mMiptg-R1      0.00000        0.06382        0.09846   
c2-b5-03_c2_0.5mMiptg-R2      0.00000        0.06276        0.09953   
c2-b5-03_c2_0.5mMiptg-R3      0.00000        0.06168        0.09855   
c2-b5-03_c2_1.0mMiptg-R1      0.00000        0.05985        0.09831   
c2-b5-03_c2_1.0mMiptg-R2      0.00000        0.06192        0.09943   
c2-b5-03_c2_1.0mMiptg-R3      0.00000        0.06243        0.09903   
c2-b5-04_c2_0.5mMiptg-R1      0.24766        0.05359        0.09996   
c2-b5-04_c2_0.5mMiptg-R2      0.24679        0.05082        0.09961   
c2-b5-04_c2_0.5mMiptg-R3      0.26410        0.05461        0.09982   
c2-b5-04_c2_1.0mMiptg-R1      0.23005        0.05175        0.09830   
c2-b5-04_c2_1.0mMiptg-R2      0.24922        0.05032        0.09907   
c2-b5-04_c2_1.0mMiptg-R3      0.27461        0.05198        0.09962   
c2-b5-07_c2-R1                0.93985        0.07363        0.12016   
c2-b5-07_c2-R2                0.92886        0.07197        0.11758   
c2-b5-07_c2-R3                0.94019        0.07101        0.11749   
c2-b5-09_c2-R1                0.14078        0.04405        0.09586   
c2-b5-09_c2-R2                0.15048        0.00000        0.09586   
c2-b5-09_c2-R3                0.14510        0.00000        0.00000   
c2-b5-11_c1-R1                0.72920        0.08681        0.12353   
c2-b5-11_c1-R2                0.74988        0.07663        0.11704   
c2-b5-11_c1-R3                0.73598        0.07596        0.12018   
c2-b5-11_c1_0.5mMiptg-R1      0.65291        0.04964        0.10754   
c2-b5-11_c1_0.5mMiptg-R2

## Merge Cycle 1 & 2 Data

In [14]:
cycle2_df[('Metadata','Batch')] += 1

In [15]:
df = pd.concat([df,cycle2_df])
#display(df.loc[df[('Metadata','Cycle')]==2]['TIR'])

In [16]:
#Remove Control Strains with RFP
df = df.drop(df.loc[df[('Metadata','Cycle')].isnull()].index)
#display(df.loc[df[('Metadata','Cycle')].isnull()])

In [17]:
#Create New Standardized Line Names
#Line Name Format C1-S9-B3-R1

df = df.reset_index()

def rename_line(x):
    
    #Get an updataed batch string
    if x[('Metadata','Batch')] < 4:
        batch_string = '1.{}'.format(int(x[('Metadata','Batch')]))
    else:
        batch_string = '2.{}'.format(int(x[('Metadata','Batch')])-1)
        
    line_name = 'C{}-S{}-B{}-R{}'.format(int(x[('Metadata','Cycle')]),
                                         int(x[('Metadata','Strain')]),
                                         batch_string,
                                         int(x[('Metadata','Replicate')]))
                             
    #Check to add IPTG for strains that do not have standard concentration
    if float(x[('Metadata','IPTG')]) > STANDARD_IPTG:
        line_name += '-IPTG{}'.format(x[('Metadata','IPTG')])
        
    if isinstance(x[('Metadata','Notes')],str):
        line_name += '-{}'.format(x[('Metadata','Notes')])
        
    return line_name
                                               
df['Line Name'] = df.apply(rename_line,axis=1)
df = df.set_index('Line Name')
display(df)

Protocol Name                       EMOPEC                       \
Measurement Type       sp|P27250|AHR_ECOLI sp|P69451|LCFA_ECOLI   
Line Name                                                         
C1-S1-B1.1-R1                          0.0                 14.8   
C1-S1-B1.1-R2                          0.0                 14.8   
C1-S2-B1.1-R1                          0.0                 92.4   
C1-S2-B1.1-R2                          0.0                 92.4   
C1-S4-B1.1-R1                          0.0                 14.8   
C1-S4-B1.1-R2                          0.0                 14.8   
C1-S8-B1.1-R1                          0.0                 92.4   
C1-S8-B1.1-R2                          0.0                 92.4   
C1-S9-B1.1-R1                          0.0                 87.6   
C1-S9-B1.1-R2                          0.0                 87.6   
C1-S11-B1.1-R1                         0.0                 92.4   
C1-S11-B1.1-R2                         0.0                 92.4   
C1-S12-B1.1-R1                         0.0                 87.6   
C1-S12-B1.1-R2                         0.0                 87.6   
C1-S15-B1.1-R1                         0.0                 14.8   
C1-S15-B1.1-R2                         0.0                 14.8   
C1-S16-B1.1-R1                         0.0                 92.4   
C1-S16-B1.1-R2                         0.0                 92.4   
C1-S22-B1.1-R1                        41.1                 92.4   
C1-S22-B1.1-R2                        41.1                 92.4   
C0-S9017-B1.1-R1                       0.0                  0.0   
C0-S9017-B1.1-R2                       0.0                  0.0   
C1-S11-B1.2-R1                         0.0                 92.4   
C1-S11-B1.2-R2                         0.0                 92.4   
C1-S11-B1.2-R3                         0.0                 92.4   
C1-S19-B1.2-R1                         0.0                 92.4   
C1-S19-B1.2-R2                         0.0                 92.4   
C1-S19-B1.2-R3                         0.0                 92.4   
C1-S25-B1.2-R1                        41.1                 14.8   
C1-S25-B1.2-R2                        41.1                 14.8   
...                                    ...                  ...   
C2-S3-B2.5-R1-IPTG0.5                  NaN                  NaN   
C2-S3-B2.5-R2-IPTG0.5                  NaN                  NaN   
C2-S3-B2.5-R3-IPTG0.5                  NaN                  NaN   
C2-S3-B2.5-R1-IPTG1.0                  NaN                  NaN   
C2-S3-B2.5-R2-IPTG1.0                  NaN                  NaN   
C2-S3-B2.5-R3-IPTG1.0                  NaN                  NaN   
C2-S4-B2.5-R1-IPTG0.5                  NaN                  NaN   
C2-S4-B2.5-R2-IPTG0.5                  NaN                  NaN   
C2-S4-B2.5-R3-IPTG0.5                  NaN                  NaN   
C2-S4-B2.5-R1-IPTG1.0                  NaN                  NaN   
C2-S4-B2.5-R2-IPTG1.0                  NaN                  NaN   
C2-S4-B2.5-R3-IPTG1.0                  NaN                  NaN   
C2-S7-B2.5-R1                          NaN                  NaN   
C2-S7-B2.5-R2                          NaN                  NaN   
C2-S7-B2.5-R3                          NaN                  NaN   
C2-S9-B2.5-R1                          NaN                  NaN   
C2-S9-B2.5-R2                          NaN                  NaN   
C2-S9-B2.5-R3                          NaN                  NaN   
C1-S11-B2.5-R1                         NaN                  NaN   
C1-S11-B2.5-R2                         NaN                  NaN   
C1-S11-B2.5-R3                         NaN                  NaN   
C1-S11-B2.5-R1-IPTG0.5                 NaN                  NaN   
C1-S11-B2.5-R2-IPTG0.5                 NaN                  NaN   
C1-S11-B2.5-R3-IPTG0.5                 NaN                  NaN   
C2-S18-B2.5-R1                         NaN                  NaN   
C2-S18-B2.5-R2                         NaN     

In [18]:
#Add Time Column (Measurement Time 27 Hours)
df[('Metadata','Time')] = 27

In [19]:
#Load RBS DataFrame
rbs_df = pd.read_csv('data/Cycle2RBSPredictions.csv')
rbs_df['Strain'] = rbs_df['ID'].apply(lambda x: int(x.split('-')[-1]))
#rbs_df = rbs_df.set_index('Strain')

#Fill in all Cycle 2 strains with Base Strain Data
#Add TIR & EMOPEC Values to dataframe for Cycle 2
find_expression = lambda x: df.loc[(df[('Metadata','Cycle')]==1)&(df[('Metadata','Strain')]==x)]['EMOPEC'].iloc[0,:]

merge_df = df.loc[(df[('Metadata','Cycle')]==1)&(df[('Metadata','Strain')].isin(rbs_df['Base Strain']))&(df[('Metadata','Batch')]<4),
                   df.columns.get_level_values(0).isin(['EMOPEC','TIR','Metadata'])].groupby([('Metadata','Strain')]).first().reset_index()

rbs_df.columns = pd.MultiIndex.from_product([['Base'],rbs_df.columns])

#Convert Columns to Numerical Values
rbs_df[('Base','Base Strain')] = rbs_df[('Base','Base Strain')].apply(lambda x: int(x))
merge_df[('Metadata','Strain')] = merge_df[('Metadata','Strain')].apply(lambda x: int(x))

#Merge TIR and EMOPEC Data into rbs dataframe
rbs_df= pd.merge(rbs_df,merge_df,how='left',left_on=[('Base','Base Strain')],right_on=[('Metadata','Strain')])

#Add updated strain TIR & Emopec to each strain
def update_rbs_df(row):
    #Update Metadata
    row[('Metadata','Cycle')] = 2
    row[('Metadata','Strain')] = row[('Base','Strain')]
    
    #Update TIR & EMOPEC
    if row[('Base','Gene')] == 'FatB':
        row[('EMOPEC','sp|Q41635|FATB_UMBCA')] = row[('Base','EMOPEC')]
        row[('TIR','sp|Q41635|FATB_UMBCA')] = row[('Base','TIR')]
        
    elif row[('Base','Gene')] == 'FadD':
        row[('EMOPEC','sp|P69451|LCFA_ECOLI')] = row[('Base','EMOPEC')]
        row[('TIR','sp|P69451|LCFA_ECOLI')] = row[('Base','TIR')]
        
    return row

#Update Dataframe
rbs_df = rbs_df.apply(update_rbs_df,axis=1)

#Drop columns that will interfere with merge (Batch/Replicate) + Base
rbs_df = rbs_df.drop(df.loc[:,[('Metadata','Batch'),('Metadata','Replicate')]].head(0).columns, axis=1)
rbs_df = rbs_df.drop(rbs_df.loc[:,rbs_df.columns.get_level_values(0)=='Base'].head(0).columns,axis=1)
#display(rbs_df)

#Merge rbs_df back into the cycle2 dataframe
df[('Metadata','Strain')] = df[('Metadata','Strain')].apply(lambda x: int(x))
rbs_df[('Metadata','Strain')] = rbs_df[('Metadata','Strain')].apply(lambda x: int(x))

#display(df['Metadata'])
#merge_df = pd.merge(df.loc[df[('Metadata','Cycle')]==2],rbs_df,how='inner',on=[('Metadata','Strain')])
#display(merge_df)

def combine_rbs(row,df):
    #display(row)
    try:
        row['EMOPEC'] = df.loc[df[('Metadata','Strain')]==row[('Metadata','Strain')]]['EMOPEC'].iloc[0]
        row['TIR'] = df.loc[df[('Metadata','Strain')]==row[('Metadata','Strain')]]['TIR'].iloc[0]
    except:
        pass
    return row

df.loc[df[('Metadata','Cycle')]==2] = df.loc[df[('Metadata','Cycle')]==2].apply(lambda x: combine_rbs(x,rbs_df),axis=1)
#display(df.loc[df[('Metadata','Cycle')]==2])

#Update All cycle 1 Strains that don't have EMOPEC & 
df.loc[(df[('Metadata','Batch')]>3)&(df['Metadata','Cycle']<2)] = df.loc[(df[('Metadata','Batch')]>3)&(df['Metadata','Cycle']<2)].apply(lambda x: combine_rbs(x,df),axis=1)

#Display Remaining TIR/EMOPEC Values that are not filled in.
display(df.loc[df[('EMOPEC','sp|P27250|AHR_ECOLI')].isnull()]['Metadata'])

Measurement Type,Batch,Cycle,IPTG,Notes,Replicate,Strain,Time
Line Name,,,,,,,


In [20]:
#Add Pathway Data to Metadata
print(max(float('NaN'),float('NaN')))
import numpy as np
import math
def get_pathway(row):
    pathway_genes = [row[('TIR','tr|A1U2T0|A1U2T0_MARHV')],row[('TIR','tr|A1U3L3|A1U3L3_MARHV')],row[('TIR','sp|Q6F7B8|ACR1_ACIAD')]]
    if max(pathway_genes) == 0:
        return 0
    elif math.isnan(max(pathway_genes)):
        return float('NaN')
    else:
        return np.argmax(pathway_genes) + 1
    
    return np.argmax(row[('')])

df[('Metadata','Pathway')] = df.apply(get_pathway,axis=1)
print(df[('Metadata','Pathway')].unique())
display(df['Metadata'])
display(df['Targeted Proteomics'])

nan
[2 1 3 0]


Measurement Type,Batch,Cycle,IPTG,Notes,Replicate,Strain,Time,Pathway
Line Name,,,,,,,,
C1-S1-B1.1-R1,1.0,1.0,0.1,NaN,1.0,1,27,2
C1-S1-B1.1-R2,1.0,1.0,0.1,NaN,2.0,1,27,2
C1-S2-B1.1-R1,1.0,1.0,0.1,NaN,1.0,2,27,2
C1-S2-B1.1-R2,1.0,1.0,0.1,NaN,2.0,2,27,2
C1-S4-B1.1-R1,1.0,1.0,0.1,NaN,1.0,4,27,2
C1-S4-B1.1-R2,1.0,1.0,0.1,NaN,2.0,4,27,2
C1-S8-B1.1-R1,1.0,1.0,0.1,NaN,1.0,8,27,2
C1-S8-B1.1-R2,1.0,1.0,0.1,NaN,2.0,8,27,2
C1-S9-B1.1-R1,1.0,1.0,0.1,NaN,1.0,9,27,2


Measurement Type,sp|P27250|AHR_ECOLI,sp|P69451|LCFA_ECOLI,sp|Q41635|FATB_UMBCA,sp|Q6F7B8|ACR1_ACIAD,tr|A1U2T0|A1U2T0_MARHV,tr|A1U3L3|A1U3L3_MARHV
Line Name,,,,,,
C1-S1-B1.1-R1,125.0,111883.0,131969.0,0.0,1800.0,159144.0
C1-S1-B1.1-R2,0.0,109006.0,155128.0,0.0,0.0,173165.0
C1-S2-B1.1-R1,530.0,106977.0,123692.0,396.0,0.0,158408.0
C1-S2-B1.1-R2,470.0,110415.0,161836.0,0.0,1744.0,155221.0
C1-S4-B1.1-R1,431.0,79192.0,114226.0,0.0,1862.0,565758.0
C1-S4-B1.1-R2,286.0,84364.0,117131.0,1352.0,0.0,579082.0
C1-S8-B1.1-R1,0.0,165598.0,113884.0,0.0,0.0,202718.0
C1-S8-B1.1-R2,0.0,169995.0,107398.0,0.0,0.0,204510.0
C1-S9-B1.1-R1,0.0,745982.0,136812.0,0.0,8828.0,207289.0


In [21]:
print(list(df.columns))

[('EMOPEC', 'sp|P27250|AHR_ECOLI'), ('EMOPEC', 'sp|P69451|LCFA_ECOLI'), ('EMOPEC', 'sp|Q41635|FATB_UMBCA'), ('EMOPEC', 'sp|Q6F7B8|ACR1_ACIAD'), ('EMOPEC', 'tr|A1U2T0|A1U2T0_MARHV'), ('EMOPEC', 'tr|A1U3L3|A1U3L3_MARHV'), ('GC-MS', 'dodecan-1-ol'), ('GC-MS', 'hexadecan-1-ol'), ('GC-MS', 'octadecan-1-ol'), ('GC-MS', 'tetradecan-1-ol'), ('HPLC', 'Acetate'), ('HPLC', 'D-Glucose'), ('HPLC', 'D-Lactate'), ('HPLC', 'Formate'), ('HPLC', 'Pyruvate'), ('HPLC', 'Succinate'), ('LC-MS', '(S)-3-Hydroxyhexadecanoyl-CoA'), ('LC-MS', '3-Phospho-D-glycerate'), ('LC-MS', 'ADP'), ('LC-MS', 'AMP'), ('LC-MS', 'ATP'), ('LC-MS', 'Acetyl-CoA'), ('LC-MS', 'Citrate'), ('LC-MS', 'D-Fructose 1,6-bisphosphate'), ('LC-MS', 'D-Fructose 6-phosphate'), ('LC-MS', 'D-Glucose 6-phosphate'), ('LC-MS', 'Decanoic acid'), ('LC-MS', 'Dihydroxyacetone phosphate'), ('LC-MS', 'Dodecanoyl-CoA (n-C12:0CoA)'), ('LC-MS', 'Flavin adenine dinucleotide oxidized'), ('LC-MS', 'Glyceraldehyde 3-phosphate'), ('LC-MS', 'Isocitrate'), ('LC-MS'

In [22]:
#Drop HPLC Measurements we don't trust!
bad_HPLC_measurements = [('HPLC', 'D-Lactate'), ('HPLC', 'Formate'), ('HPLC', 'Pyruvate'), ('HPLC', 'Succinate')]
df = df.loc[:,~df.columns.isin(bad_HPLC_measurements)]

### Write out CSV and Verify it!
Write out CSVs for the complete aggrigate data set (Which I will use).  Additionally Write out EDD specific Files for upload and use By Ajinomoto People

In [23]:
#Write out Complete Data (EDD_Ajinomoto_Data)
df.to_csv('data/EDD_Ajinomoto_Data.csv')

In [24]:
#Make sure it can be pulled in easily
df_check = pd.read_csv('data/EDD_Ajinomoto_Data.csv',header=[0, 1], index_col=0,skipinitialspace=False)
#display(df_check['Metadata'])

In [25]:
#Create Takuya CSV
tak_df = df[[('Metadata','Cycle'),('Metadata','Strain')]]
tak_df[('Metadata','JPUB')] = float('NaN')
tak_df = tak_df.groupby([('Metadata','Cycle'),('Metadata','Strain')]).size().reset_index()
tak_df.columns = ['Cycle','Strain','Count']
tak_df['JPUB'] = float('NaN')

tak_df[['Cycle','Strain','JPUB']].to_csv('data/takuya_jpub.csv',index=False)
display(tak_df)

/Users/zakcostello/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Cycle,Strain,Count,JPUB
0,0.0,9017,17,NaN
1,1.0,1,5,NaN
2,1.0,2,5,NaN
3,1.0,3,3,NaN
4,1.0,4,2,NaN
5,1.0,5,3,NaN
6,1.0,6,3,NaN
7,1.0,7,3,NaN
8,1.0,8,5,NaN
9,1.0,9,2,NaN


In [26]:
#PubChemID
#print(list(df.columns.get_level_values(0).unique()))
pubchem_df = pd.DataFrame(list(df.loc[:,df.columns.get_level_values(0).isin(['LC-MS','HPLC','GC-MS','NIMS'])].columns.get_level_values(1).unique()))

pubchem_df.columns = ['Curent Name']
pubchem_df['PubChem ID'] = float('NaN')
pubchem_df = pubchem_df.set_index('Curent Name')
pubchem_df.to_csv('data/pubchem_names.csv')
print(len(pubchem_df))
display(pubchem_df)

49


,PubChem ID
Curent Name,
dodecan-1-ol,NaN
hexadecan-1-ol,NaN
octadecan-1-ol,NaN
tetradecan-1-ol,NaN
Acetate,NaN
D-Glucose,NaN
(S)-3-Hydroxyhexadecanoyl-CoA,NaN
3-Phospho-D-glycerate,NaN
ADP,NaN


In [27]:
#Check NaN Rows for TIR/EMOPEC/METADATA/PROTEOMICS/PRODUCTION
dense_df = df.loc[:,df.columns.get_level_values(0).isin(['TIR','Targeted Proteomics','GC-MS'])]
#display(dense_df.loc[dense_df.isnull().any(axis=1)])
#display(df['Metadata'].loc[df[('Metadata','Pathway')].isnull()])